#*CLASSIFICAZIONE OGGETTI E PERSONE*

In questo script viene usata una rete SEGNET , con lo scopo di classificare le immagini del dataset-civiltà-contadina. Cercando di restituire la classe di appartenenza del oggetto che abbiamo nell'immagine. Si preleva ad esempio un immagine 86.JPG e si cerca di capire se sia un oggetto o persona. Mancano alcuni accorgimenti:
- data argomentation
- pre-addestramento
- nessuna ottimizazzione nella scelta del numero delle epoche
- nessun cross-validatione dei dati di input

Problemi nell'addestramento:
La GPU utilizza tutta la RAM del processore locale a disposizione dopo pochi minuti. Per cambiare impostazioni vai su RUNTIME>CAMBIA TIPO DI RUNTIME> TPU o tanti altri. Gpu di Colab limitata.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#SYSTEM
import os

#IMAGES
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image

#MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute

In [ ]:
# MODEL: SEGNET
def segnet(input_shape=(224, 224, 3), num_classes=2):
    # Input
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Decoder
    up3 = UpSampling2D(size=(2, 2))(pool3)
    conv_decode3 = Conv2D(256, (3, 3), activation='relu', padding='same')(up3)
    conv_decode3 = BatchNormalization()(conv_decode3)
    conv_decode3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_decode3)
    conv_decode3 = BatchNormalization()(conv_decode3)

    up2 = UpSampling2D(size=(2, 2))(conv_decode3)
    conv_decode2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv_decode2 = BatchNormalization()(conv_decode2)
    conv_decode2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv_decode2)
    conv_decode2 = BatchNormalization()(conv_decode2)

    up1 = UpSampling2D(size=(2, 2))(conv_decode2)
    conv_decode1 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv_decode1 = BatchNormalization()(conv_decode1)
    conv_decode1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_decode1)
    conv_decode1 = BatchNormalization()(conv_decode1)

    # Output
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv_decode1)

    # Modello finale
    model = Model(inputs=inputs, outputs=outputs)

    return model

# Definizione del modello SegNet
model = segnet()

# Compilazione del modello
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Visualizzazione dell'architettura del modello
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 224, 224, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0     

- SEGNET- modello della rete
- COCODatasetPearson - dataset per addestramento
- Test - dataset-civiltà-contadina

In [ ]:
!pip install pycocotools opencv-python-headless

In [ ]:
!ls /content/drive//MyDrive/dataset-civiltà-contadina/COCODatasetPerson

README.dataset.txt  README.roboflow.txt  test  train  valid


In [ ]:
train_images_dir = '/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train'
valid_images_dir = '/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/valid'

for root, dirs, files in os.walk(train_images_dir):
    print(root) # Visualizziamo la directory di tutti i file all'interno
    for name in files:
        print(os.path.join(root, name))

/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000055903_jpg.rf.5ea1ef5163463f550356e307317e2799.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000373198_jpg.rf.a6e5c2eabebf3d2d36accab765aa8a79.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000202004_jpg.rf.bf45eb065176f25c305b805727b45286.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000319615_jpg.rf.1af6102cd1dfb82b247e8edd4bf5f006.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000419265_jpg.rf.05ef29c42006dd8cc3b94236a081d500.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000269975_jpg.rf.d2588d18be0dc1465957a63b39576108.jpg
/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train/000000071499_jpg.rf.afb3f7e39f38bb78c4038daacd989660.jpg
/content/

Attenzione: L'operazione rihiede tanto tempo , si puo addestrare

In [ ]:
# Funzione per caricare le immagini da una directory
def load_images_from_directory(directory, target_size=(224, 224)):
    images = []
    labels = []
    label = 0  # etichetta fittizia per tutte le immagini
    for root, _, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, name)
                img = Image.open(img_path).convert('RGB')
                img = img.resize(target_size)
                img_array = np.array(img)
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

# Percorsi delle directory
train_images_dir = '/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/train'
valid_images_dir = '/content/drive/MyDrive/dataset-civiltà-contadina/COCODatasetPerson/valid'

# Carica le immagini e crea etichette fittizie
train_images, train_labels = load_images_from_directory(train_images_dir)
valid_images, valid_labels = load_images_from_directory(valid_images_dir)

# Stampa delle dimensioni per verificare
print(f"Train images shape: {train_images.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation images shape: {valid_images.shape}, Validation labels shape: {valid_labels.shape}")



Train images shape: (3721, 224, 224, 3), Train labels shape: (3721,)
Validation images shape: (1002, 224, 224, 3), Validation labels shape: (1002,)


In [ ]:
# Esempio di normalizzazione delle immagini
train_images_normalized = train_images / 255.0
valid_images_normalized = valid_images / 255.0

# Necessario
from keras.utils import to_categorical

train_labels_categorical = to_categorical(train_labels)
valid_labels_categorical = to_categorical(valid_labels)

In [ ]:
history = model.fit(
    train_images_normalized, train_labels_categorical,
    epochs=10,
    batch_size=32,
    validation_data=(valid_images_normalized, valid_labels_categorical)
)

La GPU utilizza tutta la RAM del processore locale a disposizione dopo pochi minuti. Per cambiare impostazioni vai su RUNTIME>CAMBIA TIPO DI RUNTIME> TPU o tanti altri. Gpu di Colab limitata.